# Utilizando OpenRouter e ChatOpenAi
Aprendendo a utilziar a Langchain com openrouter para me comunicar com a LLM Gratuita da llama

In [3]:
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

load_dotenv()

API_KEY = os.getenv("API_KEY")
if API_KEY is None:
    raise ValueError("A chave da API não foi definida no .env")

numero_dias = 7
numero_criancas = 2
atividade = 'praia'

modelo_de_prompt = PromptTemplate(
    template="""
    Crie um roteiro de viagem de {dias}, para uma família com {numero_criancas}, crianças que gostam de {atividade}
    """,

)

prompt = modelo_de_prompt.format(
    dias=numero_dias,
    numero_criancas=numero_criancas,
    atividade=atividade
)

print("Prompt: \n", prompt)

modelo = ChatOpenAI(
    openai_api_key=API_KEY,                             # Chave api da OpenRouter
    openai_api_base="https://openrouter.ai/api/v1",     # URL base da api do OpenRouter (compatível com OpenAI)
    model_name="meta-llama/llama-3-8b-instruct"         # Modelo a ser utilizado (llama 3 8bclear)
)

response = modelo.invoke(prompt)
print("Resposta: \n", response.content)

Prompt: 
 
    Crie um roteiro de viagem de 7, para uma família com 2, crianças que gostam de praia
    
Resposta: 
 Aqui está um roteiro de viagem de 7 dias para uma família com 2 crianças que gostam de praia:

**Dia 1: Chegada em Cancún, México**

* Cheguem a Cancún, uma cidade知 popular na costa leste do México, conhecida por suas praias paradisíacas e vida noturna animada.
* Instalem-se em um hotel de praia, como o Hotel Riu Cancún ou o Club Med Cancún Yucatán.
* Durante a tarde,OPTIONAL, explore a cidade e visitem o mercadinho de.Xna, um mercado localizado no coração da cidade, onde podem encontrar artesanatos e comidas típicas.
* Noite: desfrutem de uma noite de lazer em um clube infantil ou em um cinema.

**Dia 2: Praia e aventura**

* Comecem o dia com um lanche de brunch em um local de praia, como o Restaurante El Fogoncito.
* Depois, ativem o espírito de aventura com um passeio em um snorkeling ou um mergulho em uma praia como Playa Tortugas ou Playa Nizuc.
* Noite: retornem a

# Utilizando LCEL
 Utilizar o Langchain para formar uma "cadeia de chamadas" para retornar string de resposta

In [3]:
import os
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

load_dotenv()

API_KEY = os.getenv("API_KEY")
if API_KEY is None:
    raise ValueError("A chave da API não foi definida no .env")

prompt_cidade = PromptTemplate(
    template="""
    Sugira uma cidade dado o meu interesse por {interesse}.
    """,
    input_variables=["interesse"]
)

modelo = ChatOpenAI(
    openai_api_key=API_KEY,                             # Chave api da OpenRouter
    openai_api_base="https://openrouter.ai/api/v1",     # URL base da api do OpenRouter (compatível com OpenAI)
    model_name="meta-llama/llama-3-8b-instruct"         # Modelo a ser utilizado (llama 3 8bclear)
)

cadeia = prompt_cidade | modelo | StrOutputParser()

response = cadeia.invoke(
    {
        "interesse": "praias"
    }
)
print("Resposta: \n", response)

Resposta: 
 Que delícia! Com o seu interesse em praias, sugiro a cidade de Florianópolis, no Brasil!

Florianópolis é conhecida como a " Ilha de São Pedro" e é conhecida pela sua beleza natural, com mais de 42 praias ao longo de suas costas. Você encontrará praias paradisíacas, como a famosa Praia da Joaquina, Praia Mole, Praia do Campeche e muitas outras.

Além disso, a cidade tem um clima tropical, com temperaturas agradáveis durante o ano e uma vegetação lush, o que torna a área ainda mais atraente.

Alguns outros pontos turísticos para você explorar em Florianópolis são:

1. O centro histórico da cidade, com suas ruas arborizadas e edifícios antigos.
2. O Canal da Pavunas, um ponto turístico muito popular para observação de animais marinhos e golfinhos.
3. A Praça XV de Novembro, um parque alegre e aconchegante com eventos e culturas ao ar livre.
4. O Parque Tití, um refúgio natural para a vida selvagem e para os naturistas.

Florianópolis é uma cidade que oferece algo para todos, 

### Outros formatos de saída om LCEL
Podemos retornar respostas da LLM como JSON, CSV ou outros tipos de outputs.
Também é possível utilzar o debug e criar o modelo de resposta utilizando `Field` e uma classe extendida com `BaseModel`.

In [ ]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser # Para retornar como Json
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.globals import set_debug
from pydantic import Field, BaseModel
from dotenv import load_dotenv
import os

# set_debug(True)

load_dotenv()

API_KEY = os.getenv("API_KEY")
if API_KEY is None:
    raise ValueError("A chave da API não foi definida no .env")

class Destino(BaseModel):
    cidade:str = Field("A cidade recomendada para visitar")
    motivo:str = Field("Motivo pelo qual é interessante visitar essa cidade") 

parseador = JsonOutputParser(pydantic_object=Destino)

prompt_cidade = PromptTemplate(
    template="""
    Sugira uma cidade dado o meu interesse por {interesse}.

    {formato_de_saida}
    """,
    input_variables=["interesse"],
    partial_variables={"formato_de_saida": parseador.get_format_instructions()}
)

modelo = ChatOpenAI(
    openai_api_key=API_KEY,                             # Chave api da OpenRouter
    openai_api_base="https://openrouter.ai/api/v1",     # URL base da api do OpenRouter (compatível com OpenAI)
    model_name="meta-llama/llama-3-8b-instruct"         # Modelo a ser utilizado (Mistal AI)
)

cadeia = prompt_cidade | modelo | parseador

response = cadeia.invoke(
    {
        "interesse": "praias"
    }
)
print("Resposta: \n", response)

Resposta: 
 {'cidade': 'Cancún', 'motivo': 'Cancún é conhecida pelas suas praias lindas e cristalinas, com areia branca e águas azuis, além de ser um destino popular para férias.'}


## Concatendando uma cadeia com saídas diferentes

In [ ]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser # Para retornar como Json
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.globals import set_debug
from pydantic import Field, BaseModel
from dotenv import load_dotenv
import os

# set_debug(True)

load_dotenv()

API_KEY = os.getenv("API_KEY")
if API_KEY is None:
    raise ValueError("A chave da API não foi definida no .env")

class Destino(BaseModel):
    cidade:str = Field("A cidade recomendada para visitar")
    motivo:str = Field("Motivo pelo qual é interessante visitar essa cidade") 

class Restaurantes(BaseModel):
    cidade:str = Field("A cidade recomendada para visitar")
    restaurantes:str = Field("Restaurantes recomendados na cidade")

parseador_destino = JsonOutputParser(pydantic_object=Destino)
parseador_restaurantes = JsonOutputParser(pydantic_object=Restaurantes)


prompt_cidade = PromptTemplate(
    template="""
    Sugira uma cidade dado o meu interesse por {interesse}.

    {formato_de_saida}
    """,
    partial_variables={"formato_de_saida": parseador_restaurantes.get_format_instructions()}
)

# Após encontrar a cidade irá buscar por um restaurante naquela cidade
prompt_restaurantes = PromptTemplate(
    template="""
    Sugira restaurantes populares dentre locais em {cidade}.

    {formato_de_saida}
    """,
    input_variables=["interesse"],
    partial_variables={"formato_de_saida": parseador_destino.get_format_instructions()}
)

# Após encontrar o restaurante, pegará atividades na mesma cidade
prompt_cultural = PromptTemplate(
    template="Sugira atividades e locais culturais em {cidade}"
)

modelo = ChatOpenAI(
    openai_api_key=API_KEY,                             # Chave api da OpenRouter
    openai_api_base="https://openrouter.ai/api/v1",     # URL base da api do OpenRouter (compatível com OpenAI)
    model_name="meta-llama/llama-3-8b-instruct"         # Modelo a ser utilizado (Mistal AI)
)


cadeia_1 = prompt_cidade | modelo | parseador_destino
cadeia_2 = prompt_restaurantes | modelo | parseador_restaurantes
cadeia_3 = prompt_cultural | modelo | StrOutputParser()

cadeia = (cadeia_1 | cadeia_2 | cadeia_3)

response = cadeia.invoke(
    {
        "interesse": "praias"
    }
)
print("Resposta: \n", response)

Resposta: 
 1. Ponta Negra Beach: Localizada na costa de Fortaleza, capital do Ceará, é uma praia movimentada com ao redor 300 lojas mantidas por pretos.
2. Dragão do Mar Stadium: Estádio onde os times Fortaleza Football Club destaca.
